In [1]:
import sys
import pandas as pd #'1.0.4'
import scipy as sc
import numpy as np
import scipy.integrate as integrate
import matplotlib.pylab as pl
import re

In [2]:
cali = pd.read_csv("../Data/Cali_Simp.csv")
d = pd.read_csv("../Data/Final_Data.csv")

/home/xiang/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
d.head()

,Folder,h,ImageID,SourceFile,CreateDate,Keywords,Make,Model,Sequence,placeID,contact,species1,species2,contact.3,species3,Human.Presence,Sp
0,100_HH_11_5_18100_BTCF_T/,15.0,1,100_HH_11_5_18100_BTCF_T/IMG_0001.JPG,2018:05:03 15:22:40,placeID: 100,Prometheus,BTC5HDP,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100_HH_11_5_18100_BTCF_T/,15.0,2,100_HH_11_5_18100_BTCF_T/IMG_0002.JPG,2018:05:03 15:22:44,placeID: 100,Prometheus,BTC5HDP,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100_HH_11_5_18100_BTCF_T/,15.0,3,100_HH_11_5_18100_BTCF_T/IMG_0003.JPG,2018:05:03 15:22:45,placeID: 100,Prometheus,BTC5HDP,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100_HH_11_5_18100_BTCF_T/,15.0,4,100_HH_11_5_18100_BTCF_T/IMG_0004.JPG,2018:05:03 15:22:46,placeID: 100,Prometheus,BTC5HDP,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100_HH_11_5_18100_BTCF_T/,15.0,5,100_HH_11_5_18100_BTCF_T/IMG_0005.JPG,2018:05:03 15:22:46,placeID: 100,Prometheus,BTC5HDP,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
temp = d.loc[d.SourceFile.isin(cali.SourceFile),["SourceFile", "species1", "Human.Presence", "contact", "species2", "species3", "Sp"]]

In [5]:
temp["species1"].unique()

array([nan, 'Blackbird', 'Squirrel', 'Fox', 'Crow', 'Dog', 'Song Thrush',
       'Human', 'Thrush', 'Magpie', 'Hedgehog', 'Other bird', 'Mouse',
       'Wood Pigeon', 'Jay', 'Cat', 'Muntjac', 'Other mammal', 'Tit',
       'Bird', 'Rat', 'Pigeon'], dtype=object)

In [6]:
temp["species2"].unique()

array([nan, 'Squirrel', 'Human', 'Blackbird', 'Dog', 'Song Thrush',
       'Other bird', 'Fox', 'Crow', 'Magpie', 'Wood Pigeon',
       'Other mammal', 'Tit', 'Bird', 'Jay'], dtype=object)

In [7]:
temp["species3"].unique()

array([nan, 'Dog', 'Squirrel', 'Human', 'Wood Pigeon', 'Crow', 'Muntjac',
       'Other mammal', 'Magpie', 'Tit', 'Bird', 'Blackbird',
       'Song Thrush'], dtype=object)

In [8]:
temp["Human.Presence"].unique()

array([nan,  1.])

In [9]:
temp["contact"].unique()

array([nan,  1.,  2.])

In [10]:
temp["Sp"].unique()

array([nan], dtype=object)

In [11]:
temp.count()

SourceFile        54398
species1          26458
Human.Presence     1391
contact            5386
species2           5416
species3           1478
Sp                    0
dtype: int64

In [12]:
temp["species1"].value_counts()

Human           8865
Dog             5636
Squirrel        5329
Fox             1932
Blackbird       1261
Magpie          1008
Wood Pigeon      938
Hedgehog         482
Song Thrush      304
Tit              206
Crow             167
Other bird       109
Mouse             68
Thrush            62
Jay               38
Cat               19
Muntjac           19
Pigeon             6
Rat                5
Other mammal       3
Bird               1
Name: species1, dtype: int64

In [13]:
temp["Human.Presence"].value_counts()

1.0    1391
Name: Human.Presence, dtype: int64

In [14]:
#59 human presence, 96 species1: human   both: 11
85 + 48

133

In [15]:
hu = temp.loc[(temp["species1"] == 'Human') | (temp["Human.Presence"] == 1)]

In [16]:
hu.count()

SourceFile        9481
species1          9013
Human.Presence    1391
contact           2030
species2          2234
species3           533
Sp                   0
dtype: int64

In [17]:
notHu = temp.loc[(temp["species1"] != 'Human') & (temp["Human.Presence"] != 1)]

In [18]:
notHu.count()

SourceFile        44917
species1          17445
Human.Presence        0
contact            3356
species2           3182
species3            945
Sp                    0
dtype: int64

In [19]:
notHu = notHu.head(9481)

In [20]:
#temp.SourceFile.str.replace(r"", r"Human Presence: 1")

In [21]:
notHu.head()["SourceFile"]

0    100_HH_11_5_18100_BTCF_T/IMG_0001.JPG
1    100_HH_11_5_18100_BTCF_T/IMG_0002.JPG
2    100_HH_11_5_18100_BTCF_T/IMG_0003.JPG
3    100_HH_11_5_18100_BTCF_T/IMG_0004.JPG
4    100_HH_11_5_18100_BTCF_T/IMG_0005.JPG
Name: SourceFile, dtype: object

In [22]:
#cali.loc[cali["SourceFile"] == "33_HH_1_5_18_DCIM/101RECNX/IMG_0120.JPG"] 

In [23]:
hu.to_csv("../Data/SampleHu.csv", index = False)

In [24]:
notHu.to_csv("../Data/SampleNotHu.csv", index = False)

In [25]:
for j in jsonFiles:
    with open("../Data/COCO/" + j ) as f:
        data = json.load(f)
    name = j.replace("+", "/").replace("_keypoints.json", ".JPG")
    if name in hu.SourceFile.unique():#with human label
        if len(data["people"]) != 0:
            #true positive
            tp += 1
        else:
            #false negative
            fn += 1
    else:#without human label
        if len(data["people"]) == 0:
            #true negative
            tn += 1
        else:
            #false positive 
            fp += 1

NameError: name 'jsonFiles' is not defined

In [29]:
cali.insert(1, "placeID", cali.SourceFile.str.extract(r"(\d+)_"))
cali

,SourceFile,placeID,ID,CreateDate,Keywords
0,58_HH_2_5_18_BTCF/100_BTCF/IMG_0001.JPG,58,58,2018:04:19 13:09:46,placeID: 58
1,58_HH_2_5_18_BTCF/100_BTCF/IMG_0002.JPG,58,58,2018:04:19 13:09:47,placeID: 58
2,58_HH_2_5_18_BTCF/100_BTCF/IMG_0005.JPG,58,58,2018:04:19 13:09:48,placeID: 58
3,58_HH_2_5_18_BTCF/100_BTCF/IMG_0004.JPG,58,58,2018:04:19 13:09:48,placeID: 58
4,58_HH_2_5_18_BTCF/100_BTCF/IMG_0003.JPG,58,58,2018:04:19 13:09:48,placeID: 58
...,...,...,...,...,...
57629,77_HH_1_6_18_T/100_BTCF/IMG_1027.JPG,77,77,2018:05:31 12:31:38,placeID: 77
57630,77_HH_1_6_18_T/100_BTCF/IMG_1029.JPG,77,77,2018:05:31 12:31:39,placeID: 77
57631,77_HH_1_6_18_T/100_BTCF/IMG_1030.JPG,77,77,2018:05:31 12:31:40,placeID: 77
57632,77_HH_1_6_18_T/100_BTCF/IMG_1032.JPG,77,77,2018:05:31 12:31:41,placeID: 77


In [31]:
len(cali["placeID"].unique())

27

In [32]:
hu.insert(1, "placeID", hu.SourceFile.str.extract(r"(\d+)_"))
len(hu["placeID"].unique())

19